# Combine Vpop designs and upload them to jinko

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options) and Patritumab deruxtecan project
jinko.initialize()

In [ ]:
# Cookbook specific imports
import json

# Cookbook specific constants
folder_id = "965f821b-dc35-4cd8-a026-dd276ddcf536"

In [ ]:
# utils function


def merge_vpop_design(vpop_design_list):
    final_design = {
        "contents": {
            "correlations": [],
            "marginalCategoricals": [],
            "marginalDistributions": [],
        }
    }
    for vpop_design in vpop_design_list:
        final_design["contents"]["correlations"].extend(
            vpop_design["contents"]["correlations"]
        )
        final_design["contents"]["marginalCategoricals"].extend(
            vpop_design["contents"]["marginalCategoricals"]
        )
        final_design["contents"]["marginalDistributions"].extend(
            vpop_design["contents"]["marginalDistributions"]
        )
        final_design["coreVersion"] = vpop_design["coreVersion"]
    return final_design

### Download the VpopDesigns you want to merge

In this use case, both vpop designs come from the same jinko.ai folder, fell free to change it

In [ ]:
# Check that we are in the correct folder and the desired items exist
# Make a request to get all project items in the specified folder and filter out only VpopDesign items
vpop_designs = jinko.make_request(
    f"/app/v1/project-item?folderId={folder_id}&type=VpopDesign"
).json()

print("Available VPOP Designs in the folder:")
for design in vpop_designs:
    print(f"- {design['name']}")

In [ ]:
# Name of the VpopDesign to download
vpop_design_name_to_download = ["vpop design parameters", "vpop design compartments"]

# it's the element of vpop_designs whose name is vpop_design_name_to_to_download
vpop_design_ids_to_download = [
    vpop_design["coreId"]["id"]
    for vpop_design in vpop_designs
    if vpop_design["name"] in vpop_design_name_to_download
]

# Get the latest version of the desired VpopDesign
vpop_design_downloaded = []
for vpop_design_id in vpop_design_ids_to_download:
    vpop_design_downloaded.append(
        jinko.make_request(
            f"/core/v2/vpop_manager/vpop_generator/{vpop_design_id}"
        ).json()
    )

### Merge the vpop designs

In [ ]:
vpop_design_merged = merge_vpop_design(vpop_design_downloaded)

### Get the Ids of the combined model

In [ ]:
# Make a request to get all project items in the specified folder and filter out only ComputationalModel items
models = jinko.make_request(
    f"/app/v1/project-item?folderId={folder_id}&type=ComputationalModel"
).json()

print("Available Models in the folder:")
for model in models:
    print(f"- {model['name']}")

### Upload the combined VpopDesign to jinko with the attached model

In [ ]:
# Name of the model corresponding to the combined vpop
model_name = "simple tumor model"

# it's the element of vpop_designs whose name is vpop_design_name_to_to_download
model_source_id = next(
    (model["coreId"] for model in models if model["name"] == model_name), None
)

In [ ]:
# Name of the model corresponding to the combined vpop
model_name = "simple tumor model"

# it's the element of vpop_designs whose name is vpop_design_name_to_to_download
model_source_id = next(
    (model["coreId"] for model in models if model["name"] == model_name), None
)

vpop_design_merged["tag"] = "VpopGeneratorFromDesign"
vpop_design_merged["contents"]["computationalModelId"] = {
    "coreItemId": model_source_id["id"],
    "snapshotId": model_source_id["snapshotId"],
}

# https://doc.jinko.ai/api/#/paths/core-v2-vpop_manager-vpop/post
response = jinko.make_request(
    path="/core/v2/vpop_manager/vpop_generator",
    method="POST",
    data=json.dumps(vpop_design_merged),
    options={
        "name": "Combined Vpop Design",
        "folder_id": folder_id,
    },
)

project_item_info = jinko.get_project_item_info_from_response(response)
print(f"Resource link: {jinko.get_project_item_url_from_response(response)}")